# 作业八

一种用于去除**完全重复内容**的简单方法是：只保留在整个语料库中**唯一出现**的行。事实证明，这足以消除很大一部分冗余内容，例如前面提到的页眉和菜单选项。在更简单的情况下，当我们删除在其他地方被**完全重复**的行时，通常可以得到每个页面的**唯一主体内容**（例如 StackOverflow 上的问题和回答）。

为此，我们可以对语料库进行**一次遍历**，统计每一行出现的次数。然后，在**第二次遍历**中，通过只保留其唯一行来重写每个文档。

一种朴素的做法是，使用一个数据结构来保存计数器，但这样会占用与存储语料库中所有唯一行相同规模的内存。一个简单的**内存优化技巧**是：不直接使用整行文本作为键，而是使用该行的**哈希值**作为键，从而使键的大小固定（而不是依赖于行的长度）。现在你将实现这种简单的去重方法。



### 问题（exact_deduplication）：3 分

编写一个函数，接收一组输入文件路径，并对这些文件执行**精确行去重**。该函数应首先统计语料库中每一行的出现频率，并使用**哈希**来降低内存使用。

然后，通过**只保留唯一行**来重写每个文件。

**交付物（Deliverable）：**
一个执行**精确行去重**的函数。你的函数应接收两个参数：
(a) 输入文件路径列表；
(b) 一个输出目录。

该函数应将每个输入文件重写到输出目录中，**文件名保持不变**，但通过删除在输入文件集合中**出现超过一次**的行来实现内容去重。

例如，如果输入路径是 `a/1.txt` 和 `a/2.txt`，输出目录是 `b/`，那么你的函数应生成文件 `b/1.txt` 和 `b/2.txt`。

请实现适配器 **[run_exact_line_deduplication]**，并确保它能够通过以下测试：

```
uv run pytest -k test_exact_line_deduplication
```

In [ ]:
import hashlib
from collections import defaultdict
from typing import List


def _line_hash(line: str) -> str:
    """
    对一行文本计算稳定哈希值
    """
    return hashlib.md5(line.encode("utf-8")).hexdigest()


def exact_deduplication(file_paths: List[str]) -> None:
    """
    对给定文件列表执行精确行去重：
    - 第一次遍历：统计每一行（基于哈希）的全局出现次数
    - 第二次遍历：仅保留全语料中唯一出现的行，重写文件

    参数：
        file_paths: 输入文件路径列表
    """

    # ---------- 第一次遍历：统计 ----------
    hash_counter = defaultdict(int)

    for path in file_paths:
        with open(path, "r", encoding="utf-8") as f:
            for line in f:
                h = _line_hash(line)
                hash_counter[h] += 1

    # ---------- 第二次遍历：重写 ----------
    for path in file_paths:
        unique_lines = []

        with open(path, "r", encoding="utf-8") as f:
            for line in f:
                h = _line_hash(line)
                if hash_counter[h] == 1:
                    unique_lines.append(line)

        # 覆盖写回文件
        with open(path, "w", encoding="utf-8") as f:
            f.writelines(unique_lines)


# 作业九

### 3.2 MinHash + LSH 文档去重

以下是中文翻译：

---

精确去重对于删除在多个网页中逐字重复的内容很有用，但无法处理文档内容略有差异的情况。例如，考虑软件许可文档——许可文档通常是从需要填写年份和软件作者姓名的模板生成的。因此，一个 MIT 许可项目的许可文件与另一个 MIT 许可项目的许可文件内容大致相同，但它们不是**精确**重复的。为了删除这种重复的、大多是模板化的内容，我们需要模糊去重。为了高效地执行文档级模糊去重，我们将使用带有局部敏感哈希（LSH）的 minhash。

为了执行模糊去重，我们将使用文档之间相似性的特定概念：每对文档 n-gram 集合之间的 Jaccard 相似度。集合 $S$ 和 $T$ 之间的 Jaccard 相似度定义为 $|S \cap T| / |S \cup T|$。要朴素地执行模糊去重，我们可以将每个文档表示为一组 n-gram，计算所有文档对之间的 Jaccard 相似度，如果超过特定的 Jaccard 相似度阈值，则将文档对标记为重复。然而，这种方法对于大型文档集合（例如 Common Crawl）是不切实际的。此外，朴素地存储一组 n-gram 将比文档本身占用更多的内存。

**MinHashing** 为了解决内存问题，我们用**签名**替换 n-gram 文档表示集合。特别是，我们希望构建签名，使得如果我们比较两个文档的签名，就能得到文档各自 n-gram 集合之间 Jaccard 相似度的近似值。Minhash 签名满足这些属性。为了计算文档 n-gram 集合 $S = \{s_1, ..., s_n\}$ 的 minhash 签名，我们需要 $k$ 个不同的哈希函数 $h_1, ..., h_k$。每个哈希函数将一个 n-gram 映射为一个整数。给定哈希函数 $h_i$，文档 n-gram 集合 $S$ 的 minhash 为 $\text{minhash}(h_i, S) = \min(h_i(s_1), h_i(s_2), ..., h_i(s_n))$。文档 n-gram 集合 $S$ 的签名是 $\mathbb{R}^k$ 中的一个向量，其中每个元素 $i$ 包含 $S$ 在随机哈希函数 $h_i$ 下的 minhash，即 $[\text{minhash}(h_1, S), \text{minhash}(h_2, S), ..., \text{minhash}(h_k, S)]$。

结果表明，对于两个文档的 n-gram 集合 $S_1$ 和 $S_2$，集合之间的 Jaccard 相似度可以用具有相同 minhash 值的列的**比例**来近似（证明见 Leskovec et al., 2014 第 3 章第 3.3.3 节）。例如，给定文档签名 $[1,2,3,2]$ 和 $[5,2,3,4]$，n-gram 集合之间的 Jaccard 相似度近似为 $2/4$，因为这些签名的第二列和第三列具有相同的 minhash 值。

**局部敏感哈希（LSH）** 虽然 minhashing 为我们提供了一种内存高效的文档表示，保留了任意文档对之间的期望相似度，但我们仍然需要比较所有文档对才能找到相似度最大的那些。LSH 提供了一种有效的方法来将可能具有高相似度的文档分桶。为了将 LSH 应用于我们的文档签名（现在是 $\mathbb{R}^k$ 中的向量），我们将签名分成 $b$ 个带，每个带包含 $r$ 个 minhash，其中 $k = br$。例如，如果我们有 100 个元素的文档签名（由 100 个随机哈希函数生成），我们可以将其分解为



### 具体示例

假设我们有一个文档 $D_1$，其 minhash 签名为
$[1, 2, 3, 4, 5, 6]$，
另一个文档 $D_2$ 的 minhash 签名为
$[1, 2, 3, 5, 1, 2]$。

如果我们使用 **3 个 band，每个 band 含 2 个 minhash**，那么：

* $D_1$ 的第一个 band 是 $[1, 2]$，第二个 band 是 $[3, 4]$，第三个 band 是 $[5, 6]$
* $D_2$ 的第一个 band 是 $[1, 2]$，第二个 band 是 $[3, 5]$，第三个 band 是 $[1, 2]$

由于在**第一个 band** 中的哈希值完全一致（对两个文档都是 $[1, 2]$），$D_1$ 和 $D_2$ 会在该 band 下被聚到同一个桶中。
而在其他 band 中，由于哈希值不匹配，它们不会被聚到同一个桶。

不过需要注意的是：**只要文档在至少一个 band 中被聚到同一个桶里，它们就会被视为候选重复文档**，而不需要其他 band 也匹配。

---

一旦我们识别出了候选重复文档，就可以用多种方式对它们进行后处理。例如，可以计算所有候选文档对之间的 **真实 n-gram Jaccard 相似度**，并将超过某个阈值的文档对标记为重复。

---

最后，我们还需要**跨桶聚类重复文档**。
例如，假设文档 A 和 B 在某一个桶中匹配，并且它们的真实 Jaccard 相似度高于阈值；同时，文档 B 和 C 在另一个桶中匹配，且它们的真实 Jaccard 相似度也高于阈值。
那么，我们会将文档 A、B、C 视为**同一个聚类（cluster）**，并在每个聚类中**随机保留一个文档，其余删除**。

---

## 问题（MinHash 去重）：8 分

编写一个函数，该函数接收一组输入文件路径，并使用 **MinHash + LSH** 执行**模糊文档去重**。具体要求如下：

* 对给定路径列表中的每个文档计算 **MinHash 签名**
* 使用给定数量的 **band** 通过 **LSH** 识别候选重复文档
* 对候选重复文档计算 **真实的 n-gram Jaccard 相似度**
* 删除相似度超过给定阈值的文档

为提升效果（参考 *Penedo et al., 2023*），在计算 MinHash 签名和 / 或比较 Jaccard 相似度之前，应对文本进行如下规范化处理：

* 全部转换为小写
* 移除标点符号
* 规范化空白符
* 移除重音符号（accent）
* 应用 **Unicode NFD 规范化**

---

### 交付物（Deliverable）

实现一个执行模糊文档去重的函数。该函数至少应接收以下参数：

1. 输入文件路径列表
2. 用于计算 MinHash 签名的哈希函数数量
3. LSH 中使用的 band 数量
4. 用于计算 MinHash 签名的 n-gram 长度（以“词”为单位）
5. 输出目录路径

你可以假设：用于计算 MinHash 签名的哈希函数数量可以被 band 数量整除。

---

### 输出要求

你的函数应将每个输入文件写入输出目录，文件名保持不变，但**只写出以下两类文档**：

* $a$ 不是候选重复文档的文档
* $b$ 在聚类后的 bucket 中被**随机选中保留**的文档

例如，如果输入路径是 `a/1.txt` 和 `a/2.txt`，输出目录是 `b/`，那么你的函数应输出 `b/1.txt` 和 `b/2.txt`。

---

请实现适配器函数 **`run_minhash_deduplication`**，并确保它能够通过以下测试：

```bash
pytest -k test_minhash_deduplication
```

---

**注释：**
6. 关于 LSH 和 MinHash 的更深入讨论，请参考 *Leskovec et al., 2014* 第 3 章，在线版本可在 `http://infolab.stanford.edu/~ullman/mmds/ch3.pdf` 获取。
7. 这些哈希函数可以来自同一个哈希函数族，但使用不同的随机种子。例如，MurmurHash3 是一个哈希函数族，使用不同的种子可以实例化出不同的具体哈希函数。




In [3]:
import os
import re
import unicodedata
from collections import defaultdict
from itertools import combinations
import shutil  # 用于复制文件（保留元数据）
from unicodedata import normalize

def normalize_text(text: str) -> str:
    """
    预备工作 1：文本标准化（Normalization）

    目的：
    - 消除与语义无关的差异（大小写、标点、重音、空白）
    - 让“本质相同”的文本在后续 shingle 层面尽可能一致
    - 这是近似去重中“降低噪声”的关键步骤

    如果跳过此步骤：
    - 同一句话可能因为大小写或标点不同而被认为完全不相似
    """

    # 1 全部转为小写
    # WHY：避免 "Apple" 和 "apple" 被当作不同词
    text = text.lower()

    # 2 删除标点符号
    # 正则解释：
    # \w  → 字母、数字、下划线
    # \s  → 空白字符
    # [^\w\s] → 所有“非字母、非数字、非空白”的字符，即标点
    text = re.sub(r'[^\w\s]', '', text)

    # 3 规范化空白字符
    # \s+ 表示一个或多个连续空白
    # 统一替换为一个空格，并去掉首尾空格
    text = re.sub(r'\s+', ' ', text).strip()

    # 4 Unicode NFD 规范化
    # 例如：é → e + ´（把重音符号拆出来）
    text = normalize('NFD', text)

    # 5 删除所有“非间距重音符号”
    # unicodedata.category(ch) == "Mn"
    # Mn = Mark, Nonspacing（重音、变音符）
    text = "".join(
        ch for ch in text
        if unicodedata.category(ch) != "Mn"
    )

    return text

def get_shingles(text: str, n: int) -> list[str]:
    """
    预备工作 2：生成 n-gram（shingles）

    理论背景：
    - MinHash 的输入不是原始文本，而是“集合”
    - 我们用 n-gram 字符子串来近似表示文本内容

    举例：
    text = "hello", n = 3
    → {"hel", "ell", "llo"}

    为什么用 set：
    - Jaccard 相似度基于“集合”，不关心频次
    """

    shingles = set()

    # 滑动窗口提取长度为 n 的子串
    # len(text) - n + 1 确保不会越界
    for i in range(len(text) - n + 1):
        shingles.add(text[i:i+n])

    return shingles


def estimate_jaccard(sig1: list, sig2: list) -> float:
    """
    使用 MinHash 签名估算 Jaccard 相似度

    理论公式：
    J(A, B) ≈ 相等的 MinHash 行数 / 总 hash 数 K

    重要说明：
    - 这是“估算值”，不是精确 Jaccard
    - 但在 K 足够大时，期望值等于真实 Jaccard
    """

    if len(sig1) != len(sig2):
        raise ValueError("Signatures must have the same length.")

    # zip(sig1, sig2)：
    # - 同时遍历两个签名的第 i 行
    # - 判断 hash 是否相等
    matching_hashes = sum(
        1 for h1, h2 in zip(sig1, sig2) if h1 == h2
    )

    return matching_hashes / len(sig1)

def run_minhash_deduplication(
    input_files: list[os.PathLike],
    num_hashes: int,
    num_bands: int,
    ngrams: int,
    jaccard_threshold: float,
    output_directory: os.PathLike,
):
    """
    主函数：使用 MinHash + LSH 完成近似文本去重

    输入：
    - input_files         : 文档路径列表
    - num_hashes (K)      : MinHash 签名长度
    - num_bands (B)       : LSH 分段数量
    - ngrams              : shingle 的 n
    - jaccard_threshold   : 相似度阈值
    - output_directory    : 输出目录
    """
    # 第一步，先读取文件，并进行预备工作处理。
    doc_shingles = defaultdict(set)
    all_shingles = set()# 所有出现过的shingle,也就是所有文档的并集
    for file_path in input_files:
        with open(file_path, 'r', encoding='utf-8') as f:
            text = f.read()
            normalized_text = normalize_text(text)
            shingles = get_shingles(normalized_text, ngrams)
            doc_shingles[file_path] = shingles
            all_shingles.update(shingles) # 收集所有出现过的shingle
            # print(shingles)
    
    #第二步，生成MinHash签名
    # 初始化签名矩阵 M，所有值为无穷大
    # signatures[doc_id] = [inf, inf, ..., inf]，相当于按列来创建矩阵
    signatures = {doc_id: [float('inf')] * num_hashes for doc_id in doc_shingles}
    # 对每一个唯一的 shingle 计算它的 K 个哈希值
    # 这是按“行r”来计算哈希值 val_1, val_2, ...
    for shingle in all_shingles:
        # 使用不同的 salt (盐)来模拟 k 个独立的哈希函数
        hash_values = [hash(shingle + str(i)) for i in range(num_hashes)]
        # 遍历所有文档，如果文档包含这个 shingle，则更新它的签名
        for doc_id, shingles_set in doc_shingles.items():
            if shingle in shingles_set:
                # 规则：M(i, c) = min(M(i, c), val_i)
                for i in range(num_hashes):
                    signatures[doc_id][i] = min(signatures[doc_id][i], hash_values[i])

    # --- 第三步: LSH分段与分桶 ---
    r = num_hashes // num_bands # 每个band包含的hash签名数量 (rows per band)

    candidate_pairs = set()
    
    # 1. 对每一个band进行处理
    for band_index in range(num_bands):
        # buckets是一个哈希表，用于存放当前band的“哈希桶”
        # key是band的哈希值，value是落入这个桶的文档列表
        buckets = defaultdict(list)
        
        # 2. 遍历所有文档
        for doc_id, sig in signatures.items():
            # 提取当前band对应的签名部分
            start_index = band_index * r
            end_index = start_index + r
            band = tuple(sig[start_index:end_index]) #转为tuple才能作为dict的key
            # 3. 将 (band -> doc_id) 存入桶中
            buckets[band].append(doc_id)
            
        # 4. 生成候选对
        # 只要一个桶里的文档数 > 1，它们就是候选对
        for bucket_docs in buckets.values():
            if len(bucket_docs) > 1:
                # 使用itertools.combinations来生成桶内所有可能的配对
                for pair in combinations(bucket_docs, 2):
                    candidate_pairs.add(tuple(sorted(pair))) #排序后加入set，避免(a,b)和(b,a)重复

    # --- 第四步: 验证候选对并输出结果 ---    
    duplicate_pairs = []
    for doc1, doc2 in candidate_pairs:
        # 从签名矩阵中直接估算Jaccard相似度
        j_estimate = estimate_jaccard(signatures[doc1], signatures[doc2])
        
        if j_estimate >= jaccard_threshold:
            duplicate_pairs.append((doc1, doc2))

    
    # --- 第五步: 根据新要求，构建重复集群并选择要保留的文件 ---
    adj = defaultdict(list)
    for doc1, doc2 in duplicate_pairs:
        adj[doc1].append(doc2)
        adj[doc2].append(doc1)
        
    # 2. 寻找连通分量 (即重复的集群)
    clusters = []
    visited = set()
    for doc in input_files:
        if doc not in visited:
            cluster = []
            q = [doc]
            visited.add(doc)
            head = 0
            while head < len(q):
                current = q[head]
                head += 1
                cluster.append(current)
                # 仅当节点在adj中时才遍历邻居
                if current in adj:
                    for neighbor in adj[current]:
                        if neighbor not in visited:
                            visited.add(neighbor)
                            q.append(neighbor)
            clusters.append(cluster)

    # 3. 决定要保留的文件
    files_to_keep = set()
    for cluster in clusters:
        # 如果一个集群只有一个文件，说明它是唯一的
        if len(cluster) == 1:
            files_to_keep.add(cluster[0])
        else:
            # 如果是重复集群，按字母顺序排序并选择第一个作为代表
            cluster.sort()
            representative = cluster[0]
            files_to_keep.add(representative)


    # --- 第六步: 将选定的文件写入输出目录 ---
    os.makedirs(output_directory, exist_ok=True) # 创建输出目录
    
    copied_count = 0
    for file_path in files_to_keep:
        # 构建目标路径，保持文件名不变
        destination_path = os.path.join(output_directory, os.path.basename(file_path))
        shutil.copy2(file_path, destination_path) # copy2会同时复制元数据
        copied_count += 1    

    return None

我们可以生成测试文档来测试去重函数

In [1]:
import os
import random
import string
from pathlib import Path
from typing import List

# -----------------------------
# 基础词表
# -----------------------------
VOCAB = [
    "machine", "learning", "data", "model", "training", "evaluation",
    "system", "performance", "algorithm", "network", "representation",
    "optimization", "feature", "pipeline", "inference", "embedding",
    "similarity", "hashing", "document", "text"
]

TEMPLATE = """
MIT License

Copyright (c) {year} {author}

Permission is hereby granted, free of charge, to any person obtaining a copy
of this software and associated documentation files (the "Software"), to deal
in the Software without restriction, including without limitation the rights
to use, copy, modify, merge, publish, distribute, sublicense, and/or sell
copies of the Software.

This project focuses on {topic} and demonstrates {method}.
"""

# -----------------------------
# 文档生成函数
# -----------------------------
def random_sentence(min_len=5, max_len=12):
    length = random.randint(min_len, max_len)
    return " ".join(random.choice(VOCAB) for _ in range(length))


def generate_random_document(num_sentences=10):
    return "\n".join(random_sentence() for _ in range(num_sentences))


def generate_template_document(author, year, topic, method):
    return TEMPLATE.format(
        author=author,
        year=year,
        topic=topic,
        method=method
    )


def perturb_text(text: str, prob=0.2):
    """
    对文本做轻微扰动：
    - 随机大小写
    - 随机插入词
    - 随机删除词
    """
    words = text.split()
    new_words = []

    for w in words:
        if random.random() < prob:
            action = random.choice(["upper", "drop", "insert"])
            if action == "upper":
                w = w.upper()
            elif action == "drop":
                continue
            elif action == "insert":
                new_words.append(random.choice(VOCAB))
        new_words.append(w)

    return " ".join(new_words)


# -----------------------------
# 主生成逻辑
# -----------------------------
def generate_test_corpus(
    output_dir: str,
    num_random_docs=10,
    num_template_groups=5,
    perturbations_per_template=3
):
    output_dir = Path(output_dir)
    output_dir.mkdir(parents=True, exist_ok=True)

    doc_id = 0

    # 1. 完全随机文档（不应被去重）
    for _ in range(num_random_docs):
        text = generate_random_document()
        (output_dir / f"random_{doc_id}.txt").write_text(text, encoding="utf-8")
        doc_id += 1

    # 2. 模板 + 扰动文档（应被去重）
    for g in range(num_template_groups):
        base = generate_template_document(
            author=f"Author_{g}",
            year=2020 + g,
            topic=random_sentence(3, 5),
            method=random_sentence(3, 5)
        )

        base_path = output_dir / f"template_{g}_base.txt"
        base_path.write_text(base, encoding="utf-8")

        for i in range(perturbations_per_template):
            perturbed = perturb_text(base, prob=0.15)
            (output_dir / f"template_{g}_var_{i}.txt").write_text(
                perturbed, encoding="utf-8"
            )

    print(f"Generated test corpus in: {output_dir.resolve()}")
generate_test_corpus(
    output_dir="test_docs",
    num_random_docs=20,
    num_template_groups=5,
    perturbations_per_template=4
)


Generated test corpus in: /home/kangkang/my_project/CS336-Chinese-co-construction/coursework/Assignment4_Data/cs336_data/test_docs


进行测试

In [15]:
doc_dir = 'test_docs'
input_files = [
    os.path.join(doc_dir, fname)
    for fname in os.listdir(doc_dir)
    if fname.endswith('.txt')
]
run_minhash_deduplication(input_files= input_files, num_hashes= 100, num_bands= 20, ngrams= 5, jaccard_threshold=0.8,output_directory='deduplicated_output')


其中random_开头的应该全部留下，template_每个数字留下一份

# 作业 十 语言建模的数据过滤

既然我们已经实现了多种用于过滤网络爬虫数据的基元，现在让我们将其付诸实践，生成一些语言模型训练数据。    

在这部分作业中，你的目标是过滤一批CC WET文件，以生成语言模型训练数据。我们在 `/data/CC/CC*.warc.wet.gz` 位置为你准备了5000个WET文件作为起点。

特别是，你的目标是过滤CC数据转储，创建语言模型数据，使得训练后的Transformer语言模型在Paloma基准测试的C4 100个领域子集上最小化验证困惑度 [Magnusson et al., 2023]。**你不应修改模型架构或训练过程**，因为目标是构建最佳的**数据**。该数据集包含C4语言建模数据集中100个最常见领域的样本 [Raffel et al., 2020]。我们在Together集群的 `/data/paloma/tokenized_paloma_c4_100_domains_validation.bin` 位置放置了该数据的副本（使用GPT-2分词器进行分词）——你可以查看这些数据以了解其样貌。你可以使用以下代码加载：

```python
import numpy as np
data = np.fromfile(
    "/data/paloma/tokenized_paloma_c4_100_domains_validation.bin",
    dtype=np.uint16
)

from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("gpt2")
print(tokenizer.decode(data[0:2000]))
```

给定你过滤后的数据集，你将在此数据上训练一个GPT-2 small形状的模型，进行20万次迭代，并评估其在C4 100上的困惑度。

我们注意到，**你允许**使用Paloma验证数据来构建过滤器或分类器以处理CC WET文件，但**不允许**将验证数据直接复制到你的训练数据中。语言模型不应看到验证集中的任何数据。

即使是5000个WET文件也是大量的数据，约375GB的压缩文本。为了高效处理这些数据，我们建议尽可能使用多进程处理。特别是，你可能会发现Python的 `concurrent.futures` 或 `multiprocessing` API很有帮助。下面，我们展示了一个使用 `concurrent.futures` 在多个进程中并行化函数的简单示例：

```python
import concurrent.futures
import os

from tqdm import tqdm

def process_single_wet_file(input_path: str, output_path: str):
    # TODO: 读取输入路径，处理输入，并将输出写入output_path
    return output_path

# 设置执行器
num_cpus = len(os.sched_getaffinity(0))
executor = concurrent.futures.ProcessPoolExecutor(max_workers=num_cpus)
wet_filepaths = ["a.warc.wet.gz", "b.warc.wet.gz", "c.warc.wet.gz"]
output_directory_path = "/path/to/output_directory/"

futures = []
for wet_filepath in wet_filepaths:
    # 对于每个warc.wet.gz文件路径，向执行器提交一个作业并获取一个future
    wet_filename = str(pathlib.Path(wet_filepath).name)
    future = executor.submit(
        process_single_wet_file,
        wet_filepath,
        os.path.join(output_directory_path, wet_filename)
    )
    # 存储futures
    futures.append(future)

# 在任务完成时遍历已完成的futures，使用进度条来跟踪进度
for future in tqdm(
    concurrent.futures.as_completed(futures),
    total=len(wet_filepaths),
):
    output_file = future.result()
    print(f"输出文件已写入: {output_file}")
```

为了在Slurm集群上并行化你的数据处理，你可以使用 `submitit` 包，它提供了 `concurrent.futures` 的直接替代方案，可以处理在指定Slurm分区上启动作业和收集结果：
以下是中文翻译：

---

```python
import os

import submitit
from tqdm import tqdm

def process_single_wet_file(input_path: str, output_path: str):
    # TODO: 读取输入路径，处理输入，并将输出写入output_path
    return output_path

# 设置submitit执行器
executor = submitit.AutoExecutor(folder="slurm_logs")
max_simultaneous_jobs = 16
wet_filepaths = ["a.warc.wet.gz", "b.warc.wet.gz", "c.warc.wet.gz"]
output_directory_path = "/path/to/output_directory/"
# 配置submitit启动的每个作业的参数
executor.update_parameters(
    slurm_array_parallelism=max_simultaneous_jobs,
    timeout_min=15,
    mem_gb=2,
    cpus_per_task=2,
    slurm_account="student",
    slurm_partition="a4-cpu",
    slurm_qos="a4-cpu-qos",
)
futures = []
# 使用executor.batch()上下文管理器将所有作业分组到一个Slurm数组中
with executor.batch():
    for wet_filepath in wet_filepaths:
        # 对于每个WARC文件路径，向执行器提交一个作业并获取一个future
        wet_filename = str(pathlib.Path(wet_filepath).name)
        future = executor.submit(
            process_single_wet_file,
            wet_filepath,
            os.path.join(output_directory_path, wet_filename)
        )
```

$^8$ https://github.com/facebookincubator/submitit

```python
        # 存储futures
        futures.append(future)

# 使用tqdm显示进度
for future in tqdm(
    submitit.helpers.as_completed(futures),
    total=len(wet_filepaths),
):
    output_file = future.result()
    print(f"输出文件已写入: {output_file}")
```

如你所见，使用 submitit 与内置的 concurrent.futures API 时代码非常相似。主要区别在于：(1) 需要配置 submitit 执行器参数，以便它知道在哪里提交 Slurm 作业以及每个作业的资源规格；(2) 使用 `executor.batch()` 将所有 Slurm 作业分组到一个作业"数组"中（而不是 len(wet_filepaths) 个单独的作业），这样可以最小化 Slurm 调度器的负载；以及 (3) 在收集结果时使用 `submitit.helpers.as_completed`。

我们还建议使用 **fastwarc** 库来遍历每个 WET 文件中的记录，以及 **tldextract** 库来从 URL 中提取域名以进行过滤。特别是，以下类可能会有帮助：

```python
from fastwarc.warc import ArchiveIterator, WarcRecordType
from tldextract import TLDExtract
```

---

### 问题（filter_data）：6分

**(a)** 编写一个脚本，从 Common Crawl WET 文件集合中过滤语言建模数据（位于 Together 集群的 `/data/CC/CC*.warc.wet.gz`）。你可以自由应用在本作业前面部分实现的任何基元，也可以自由探索其他过滤器和数据生成方法（例如，基于 n-gram 语言模型困惑度的过滤）。你的目标是生成数据，使得在该数据上训练后，能够最小化在 Paloma 基准测试 C4 100 个领域子集上的困惑度。

再次说明，我们注意到**你允许**使用 Paloma 验证数据来构建过滤器或分类器以处理 CC WET 文件，但**不允许**将验证数据直接复制到你的训练数据中。

你的脚本应报告每个使用的过滤器保留的样本数量，以便你了解过滤器对最终输出数据的贡献。

**交付物**：一个脚本（或脚本序列），并行过滤提供的 CC WET 文件以生成语言建模数据。一份书面说明，阐述每个过滤步骤去除的被丢弃样本的比例。

**(b)** 过滤 5,000 个 WET 文件需要多长时间？过滤整个 Common Crawl 数据转储（100,000 个 WET 文件）需要多长时间？

**交付物**：数据过滤管道的运行时间。


---

当然我们无法调用在斯坦福服务器上的文件，我们可以下载几个文件用来跑通程序，wet.paths.gz是下载文件的目录，可以通过目录下载想要的文件，只需替换网址即可。

In [18]:
!wget https://data.commoncrawl.org/crawl-data/CC-MAIN-2025-51/wet.paths.gz

--2026-01-28 21:04:39--  https://data.commoncrawl.org/crawl-data/CC-MAIN-2025-51/wet.paths.gz
Connecting to 127.0.0.1:7890... connected.
Proxy request sent, awaiting response... 200 OK
Length: 200800 (196K) [binary/octet-stream]
Saving to: ‘wet.paths.gz’

wet.paths.gz        100%[===================>] 196.09K   349KB/s    in 0.6s    

2026-01-28 21:04:40 (349 KB/s) - ‘wet.paths.gz’ saved [200800/200800]



In [19]:
!gunzip wet.paths.gz

In [20]:
!head -n 10 wet.paths

crawl-data/CC-MAIN-2025-51/segments/1764871306713.64/wet/CC-MAIN-20251204191828-20251204221828-00000.warc.wet.gz
crawl-data/CC-MAIN-2025-51/segments/1764871306713.64/wet/CC-MAIN-20251204191828-20251204221828-00001.warc.wet.gz
crawl-data/CC-MAIN-2025-51/segments/1764871306713.64/wet/CC-MAIN-20251204191828-20251204221828-00002.warc.wet.gz
crawl-data/CC-MAIN-2025-51/segments/1764871306713.64/wet/CC-MAIN-20251204191828-20251204221828-00003.warc.wet.gz
crawl-data/CC-MAIN-2025-51/segments/1764871306713.64/wet/CC-MAIN-20251204191828-20251204221828-00004.warc.wet.gz
crawl-data/CC-MAIN-2025-51/segments/1764871306713.64/wet/CC-MAIN-20251204191828-20251204221828-00005.warc.wet.gz
crawl-data/CC-MAIN-2025-51/segments/1764871306713.64/wet/CC-MAIN-20251204191828-20251204221828-00006.warc.wet.gz
crawl-data/CC-MAIN-2025-51/segments/1764871306713.64/wet/CC-MAIN-20251204191828-20251204221828-00007.warc.wet.gz
crawl-data/CC-MAIN-2025-51/segments/1764871306713.64/wet/CC-MAIN-20251204191828-20251204221828-0

In [21]:
!wget https://data.commoncrawl.org/crawl-data/CC-MAIN-2025-51/segments/1764871306713.64/wet/CC-MAIN-20251204191828-20251204221828-00000.warc.wet.gz
!wget https://data.commoncrawl.org/crawl-data/CC-MAIN-2025-51/segments/1764871306713.64/wet/CC-MAIN-20251204191828-20251204221828-00001.warc.wet.gz
!wget https://data.commoncrawl.org/crawl-data/CC-MAIN-2025-51/segments/1764871306713.64/wet/CC-MAIN-20251204191828-20251204221828-00002.warc.wet.gz

--2026-01-28 21:30:10--  https://data.commoncrawl.org/crawl-data/CC-MAIN-2025-51/segments/1764871306713.64/wet/CC-MAIN-20251204191828-20251204221828-00000.warc.wet.gz
Connecting to 127.0.0.1:7890... connected.
Proxy request sent, awaiting response... 200 OK
Length: 75410076 (72M) [application/octet-stream]
Saving to: ‘CC-MAIN-20251204191828-20251204221828-00000.warc.wet.gz’

CC-MAIN-20251204191 100%[===================>]  71.92M  10.4MB/s    in 8.3s    

2026-01-28 21:30:19 (8.69 MB/s) - ‘CC-MAIN-20251204191828-20251204221828-00000.warc.wet.gz’ saved [75410076/75410076]

--2026-01-28 21:30:20--  https://data.commoncrawl.org/crawl-data/CC-MAIN-2025-51/segments/1764871306713.64/wet/CC-MAIN-20251204191828-20251204221828-00001.warc.wet.gz
Connecting to 127.0.0.1:7890... connected.
Proxy request sent, awaiting response... 200 OK
Length: 76619390 (73M) [application/octet-stream]
Saving to: ‘CC-MAIN-20251204191828-20251204221828-00001.warc.wet.gz’

CC-MAIN-20251204191 100%[===================

In [22]:
!wget https://data.commoncrawl.org/crawl-data/CC-MAIN-2025-51/segments/1764871306713.64/wet/CC-MAIN-20251204191828-20251204221828-00003.warc.wet.gz
!wget https://data.commoncrawl.org/crawl-data/CC-MAIN-2025-51/segments/1764871306713.64/wet/CC-MAIN-20251204191828-20251204221828-00004.warc.wet.gz
!wget https://data.commoncrawl.org/crawl-data/CC-MAIN-2025-51/segments/1764871306713.64/wet/CC-MAIN-20251204191828-20251204221828-00005.warc.wet.gz

--2026-01-28 21:32:23--  https://data.commoncrawl.org/crawl-data/CC-MAIN-2025-51/segments/1764871306713.64/wet/CC-MAIN-20251204191828-20251204221828-00003.warc.wet.gz
Connecting to 127.0.0.1:7890... connected.
Proxy request sent, awaiting response... 200 OK
Length: 74965696 (71M) [application/octet-stream]
Saving to: ‘CC-MAIN-20251204191828-20251204221828-00003.warc.wet.gz’

CC-MAIN-20251204191 100%[===================>]  71.49M  11.3MB/s    in 7.5s    

2026-01-28 21:32:32 (9.54 MB/s) - ‘CC-MAIN-20251204191828-20251204221828-00003.warc.wet.gz’ saved [74965696/74965696]

--2026-01-28 21:32:32--  https://data.commoncrawl.org/crawl-data/CC-MAIN-2025-51/segments/1764871306713.64/wet/CC-MAIN-20251204191828-20251204221828-00004.warc.wet.gz
Connecting to 127.0.0.1:7890... connected.
Proxy request sent, awaiting response... 200 OK
Length: 75273977 (72M) [application/octet-stream]
Saving to: ‘CC-MAIN-20251204191828-20251204221828-00004.warc.wet.gz’

CC-MAIN-20251204191 100%[===================

In [51]:
%%writefile extracted_data.py

#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
从 Common Crawl WET 文件中提取训练数据并使用 FastText 质量分类器
支持：文件级 + 记录级 双进度条，实时跳过统计，支持 Ctrl-C 中断
"""

import json
import gzip
import signal
import sys
import random
from pathlib import Path
from typing import Optional, List, Tuple,  Dict, Any
from tqdm import tqdm

# 如果脚本在子目录，把项目根加入 PYTHONPATH
sys.path.insert(0, str(Path(__file__).parent.parent))
from filter import (
    run_mask_emails,
    run_mask_ips,
    run_mask_phone_numbers,
    run_gopher_quality_filter,
    run_extract_text_from_html_bytes,
)

import fasttext


# -------------------- 配置 -------------------- #
FASTTEXT_MODEL_PATH = "quality_classifier.bin"  # 你的 FastText 模型路径


# -------------------- 工具函数 -------------------- #
def desensitize_text(text: str) -> str:
    """邮箱 / 电话 / IP 脱敏"""
    text, _ = run_mask_emails(text)
    text, _ = run_mask_phone_numbers(text)
    text, _ = run_mask_ips(text)
    return text


def process_text(text: str) -> Optional[str]:
    """脱敏 + 单行化 + 基础过滤"""
    if not text or not text.strip():
        return None
    
    text = text.strip()
    
    # 基础长度过滤（与 Gopher 类似）
    if len(text) < 200:  # 太短不要
        return None
    if len(text) > 100_000:  # 太长截断
        text = text[:100_000]
    
    # 脱敏处理
    text = desensitize_text(text)
    
    # 单行化（FastText 格式要求）
    return ' '.join(text.split())


def classify_quality(model, text: str) -> Tuple[str, float]:
    """
    使用 FastText 模型进行质量分类
    返回: (标签, 置信度分数)
    """
    # FastText 要求输入不能包含换行符
    clean_text = text.replace('\n', ' ').strip()
    if not clean_text:
        return ("unknown", 0.0)
    
    try:
        prediction = model.predict(clean_text)
        label = prediction[0][0].replace('__label__', '')
        score = prediction[1][0]
        return (label, score)
    except Exception as e:
        return ("error", 0.0)


def extract_text_from_wet_record(record_lines: List[str]) -> Optional[str]:
    """
    从 WET 记录的行列表中提取文本内容
    
    WET 格式：
    - 以 URL 行开始
    - 然后是 WARC-Header 元数据
    - 空行后是实际内容
    """
    if not record_lines:
        return None
    
    # 找到第一个空行后的内容
    content_start = 0
    for i, line in enumerate(record_lines):
        if line.strip() == '':
            content_start = i + 1
            break
    
    # 提取内容部分
    content = '\n'.join(record_lines[content_start:]).strip()
    
    # 基础过滤
    if len(content) < 50:  # 太短
        return None
    if len(content.split()) < 10:  # 词数太少
        return None
    
    return content


def parse_wet_file(file_obj) -> List[List[str]]:
    """
    解析 WET 文件，返回记录列表（每个记录是一个行列表）
    
    WET 文件格式：记录之间用 "WARC/1.0" 分隔
    """
    records = []
    current_record = []
    
    for line in file_obj:
        line = line.decode('utf-8', errors='ignore')
        
        # 新记录开始
        if line.startswith('WARC/1.0'):
            if current_record:
                records.append(current_record)
            current_record = [line]
        else:
            if current_record is not None:
                current_record.append(line)
    
    # 添加最后一个记录
    if current_record:
        records.append(current_record)
    
    return records


# -------------------- 核心提取 -------------------- #
def extract_samples_from_wet(
    wet_paths: List[str],
    target_count: int,
    model_path: str = FASTTEXT_MODEL_PATH,
    quality_threshold: float = 0.5,  # 质量分数阈值
    random_seed: int = 42,
) -> Tuple[List[str], List[str], dict]:
    """
    多文件 WET → 提取训练样本并使用 FastText 分类
    
    返回: (高质量样本列表, 低质量样本列表, 统计信息字典)
    """
    random.seed(random_seed)
    
    # 加载 FastText 模型
    print(f"加载 FastText 模型: {model_path}")
    try:
        model = fasttext.load_model(model_path)
    except Exception as e:
        print(f"模型加载失败: {e}")
        sys.exit(1)
    
    hq_samples: List[str] = []  # 高质量
    lq_samples: List[str] = []  # 低质量
    stats = {
        "total_records": 0,
        "processed": 0,
        "skipped_empty": 0,
        "skipped_short": 0,
        "hq_count": 0,
        "lq_count": 0,
        "error": 0,
    }

    # 文件级进度条
    pbar_files = tqdm(wet_paths, desc="WET文件", unit="file", position=0)

    for wet_path in pbar_files:
        # 检查是否已达到目标
        total_collected = len(hq_samples) + len(lq_samples)
        if total_collected >= target_count * 2:  # 两种质量都收集够
            pbar_files.write("已达到目标数量，提前结束")
            break
        
        if not Path(wet_path).exists():
            pbar_files.write(f"文件不存在: {wet_path}")
            continue

        # 解析 WET 文件
        try:
            with gzip.open(wet_path, 'rb') as f:
                records = parse_wet_file(f)
        except Exception as e:
            pbar_files.write(f"解析失败 {wet_path}: {e}")
            continue
        
        total_records = len(records)
        stats["total_records"] += total_records
        pbar_files.set_postfix(
            file=Path(wet_path).name, 
            total_records=total_records,
            hq=len(hq_samples),
            lq=len(lq_samples)
        )

        # 记录级进度条
        for record_lines in tqdm(
            records,
            total=total_records,
            desc="记录",
            unit="rec",
            position=1,
            leave=False,
        ):
            # 检查是否已收集足够
            if len(hq_samples) >= target_count and len(lq_samples) >= target_count:
                break
            
            # 提取文本
            text = extract_text_from_wet_record(record_lines)
            if text is None:
                stats["skipped_empty"] += 1
                continue
            
            # 处理文本
            processed = process_text(text)
            if processed is None:
                stats["skipped_short"] += 1
                continue
            
            stats["processed"] += 1
            
            # FastText 质量分类
            label, score = classify_quality(model, processed)
            
            # 根据分类结果和阈值决定保留
            if label == "hq" and score >= quality_threshold:
                if len(hq_samples) < target_count:
                    hq_samples.append(processed)
                    stats["hq_count"] += 1
            elif label == "lq" and score >= quality_threshold:
                if len(lq_samples) < target_count:
                    lq_samples.append(processed)
                    stats["lq_count"] += 1
            else:
                stats["error"] += 1

        # 更新文件级进度条信息
        pbar_files.set_postfix(
            file=Path(wet_path).name,
            hq=len(hq_samples),
            lq=len(lq_samples),
            processed=stats["processed"]
        )

    return hq_samples, lq_samples, stats


# -------------------- 辅助函数 -------------------- #
def find_wet_files(datasets_dir: Path) -> List[str]:
    """扫描目录下所有 .wet.gz 文件"""
    patterns = ["*.warc.wet.gz"]
    files = []
    for pattern in patterns:
        files.extend(datasets_dir.glob(pattern))
    return sorted(str(f) for f in files if f.exists())



def save_samples(hq_samples: List[str], lq_samples: List[str], output_dir: Path):
    """保存样本为 JSON 格式"""
    output_dir.mkdir(parents=True, exist_ok=True)
    
    hq_path = output_dir / "train_hq.jsonl"
    lq_path = output_dir / "train_lq.jsonl"
    combined_path = output_dir / "train_combined.jsonl"
    metadata_path = output_dir / "metadata.json"
    
    # 保存高质量样本 (JSON Lines 格式)
    with open(hq_path, 'w', encoding='utf-8') as f:
        for text in hq_samples:
            record = {
                "text": text,
                "label": "hq",
                "quality_score": 1.0
            }
            f.write(json.dumps(record, ensure_ascii=False) + '\n')
    
    # 保存低质量样本 (JSON Lines 格式)
    with open(lq_path, 'w', encoding='utf-8') as f:
        for text in lq_samples:
            record = {
                "text": text,
                "label": "lq",
                "quality_score": 0.0
            }
            f.write(json.dumps(record, ensure_ascii=False) + '\n')
    
    # 保存合并文件 (JSON Lines 格式)
    with open(combined_path, 'w', encoding='utf-8') as f:
        for text in hq_samples:
            record = {
                "text": text,
                "label": "hq",
                "quality_score": 1.0
            }
            f.write(json.dumps(record, ensure_ascii=False) + '\n')
        for text in lq_samples:
            record = {
                "text": text,
                "label": "lq",
                "quality_score": 0.0
            }
            f.write(json.dumps(record, ensure_ascii=False) + '\n')
    
    # 保存元数据
    metadata = {
        "total_samples": len(hq_samples) + len(lq_samples),
        "hq_samples": len(hq_samples),
        "lq_samples": len(lq_samples),
        "format": "jsonl",
        "fields": ["text", "label", "quality_score"]
    }
    with open(metadata_path, 'w', encoding='utf-8') as f:
        json.dump(metadata, f, indent=2, ensure_ascii=False)
    
    return hq_path, lq_path, combined_path, metadata_path

# -------------------- 主入口 -------------------- #
def main():
    script_dir = Path(__file__).parent
    datasets_dir = script_dir  # 或改为你的数据目录
    output_dir = script_dir / "cc_output"
    
    # 配置
    target_count = 10_0000  # 每种质量的目标数量
    quality_threshold = 0.8  # FastText 置信度阈值
    
    # 查找 WET 文件
    wet_paths = find_wet_files(datasets_dir)
    if not wet_paths:
        print(f"未找到任何 .wet.gz 文件 in {datasets_dir}")
        print("支持的格式: *.wet.gz, *.warc.wet.gz")
        return
    
    print(f"找到 {len(wet_paths)} 个 WET 文件")
    for p in wet_paths[:5]:  # 只显示前5个
        print(f"  - {Path(p).name}")
    if len(wet_paths) > 20:
        print(f"  ... 等共 {len(wet_paths)} 个文件")
    
    # 检查模型
    if not Path(FASTTEXT_MODEL_PATH).exists():
        print(f"错误: FastText 模型不存在: {FASTTEXT_MODEL_PATH}")
        return
    
    # 提取样本
    print(f"\n开始提取样本...")
    print(f"目标: {target_count} 高质量 + {target_count} 低质量")
    print(f"质量阈值: {quality_threshold}")
    
    hq_samples, lq_samples, stats = extract_samples_from_wet(
        wet_paths=wet_paths,
        target_count=target_count,
        quality_threshold=quality_threshold,
    )
    
    # 保存结果
    if hq_samples or lq_samples:
        hq_path, lq_path, combined_path,_ = save_samples(hq_samples, lq_samples, output_dir)
        
        print(f"\n{'='*50}")
        print("处理完成!")
        print(f"{'='*50}")
        print(f"总记录数: {stats['total_records']}")
        print(f"成功处理: {stats['processed']}")
        print(f"  - 跳过(空/解析失败): {stats['skipped_empty']}")
        print(f"  - 跳过(太短): {stats['skipped_short']}")
        print(f"  - 分类错误: {stats['error']}")
        print(f"\n收集样本:")
        print(f"  - 高质量 (hq): {len(hq_samples)} / {target_count}")
        print(f"  - 低质量 (lq): {len(lq_samples)} / {target_count}")
        print(f"\n保存位置:")
        print(f"  - 高质量: {hq_path}")
        print(f"  - 低质量: {lq_path}")
        print(f"  - 合并: {combined_path}")
    else:
        print("\n没有提取到任何样本")


if __name__ == "__main__":
    # 支持 Ctrl-C 中断
    signal.signal(signal.SIGINT, lambda *_: (print("\n用户中断，退出"), sys.exit(0)))
    main()

Overwriting extracted_data.py


In [52]:
!python extracted_data.py

找到 6 个 WET 文件
  - CC-MAIN-20251204191828-20251204221828-00000.warc.wet.gz
  - CC-MAIN-20251204191828-20251204221828-00001.warc.wet.gz
  - CC-MAIN-20251204191828-20251204221828-00002.warc.wet.gz
  - CC-MAIN-20251204191828-20251204221828-00003.warc.wet.gz
  - CC-MAIN-20251204191828-20251204221828-00004.warc.wet.gz

开始提取样本...
目标: 100000 高质量 + 100000 低质量
质量阈值: 0.8
加载 FastText 模型: quality_classifier.bin
WET文件:   0%| | 0/6 [00:03<?, ?file/s, file=CC-MAIN-20251204191828-202512042218
记录: 100%|███████████████████████████▉| 21598/21640 [00:24<00:00, 1104.22rec/s]
WET文件:  17%|▏| 1/6 [00:31<02:19, 27.88s/file, file=CC-MAIN-20251204191828-2025  
记录: 100%|████████████████████████████▊| 21115/21214 [00:24<00:00, 946.37rec/s]
WET文件:  33%|▎| 2/6 [00:58<01:50, 27.69s/file, file=CC-MAIN-20251204191828-2025  
记录: 100%|████████████████████████████▉| 21337/21386 [00:23<00:00, 952.14rec/s]
WET文件:  50%|▌| 3/6 [01:25<01:21, 27.27s/file, file=CC-MAIN-20251204191828-2025  
记录: 100%|███████████████████████████▊| 

In [26]:
import fasttext
import json
FASTTEXT_MODEL_PATH = "quality_classifier.bin" 
model = fasttext.load_model(FASTTEXT_MODEL_PATH )
def classify_quality(model, text: str):
    """
    使用 FastText 模型进行质量分类
    返回: (标签, 置信度分数)
    """
    # FastText 要求输入不能包含换行符
    clean_text = text.replace('\n', ' ').strip()
    if not clean_text:
        return ("unknown", 0.0)
    
    try:
        prediction = model.predict(clean_text)
        label = prediction[0][0].replace('__label__', '')
        score = prediction[1][0]
        return (label, score)
    except Exception as e:
        return ("error", 0.0)



In [ ]:
path = '/home/kangkang/my_project/CS336-Chinese-co-construction/coursework/Assignment4_Data/cs336_data/cc_output/train_lq.jsonl'
with open(path,'r') as f:
    nums = 0
    for line in f:
        data = json.loads(line)
        print(data['text'])
        print(classify_quality(model,data['text']))
        nums +=1
        if nums ==100:
            break

        
    

Software-Info: ia-web-commons.3.0.3-SNAPSHOT-20251203080200 Extracted-Date: Wed, 17 Dec 2025 15:53:56 GMT robots: checked via crawler-commons 1.7-SNAPSHOT (https://github.com/crawler-commons/crawler-commons) isPartOf: CC-MAIN-2025-51 operator: Common Crawl Admin (|||EMAIL_ADDRESS|||) description: Wide crawl of the web for December 2025 publisher: Common Crawl
('lq', 1.0000091791152954)
下载中心 - 传世家谱编纂服务有限公司 || 您好，欢迎访问传世家谱编纂服务有限公司官方网站，我们将竭诚为您服务！ 加入收藏 | 常用网址 | 联系我们 网站首页 关于我们 组织机构 网络家谱 综合新闻 谈古论今 资料下载 在线留言 制作流程 修谱指南 联系我们 今天是： 下载中心 || 您的当前位置：首页 - 下载中心 信息标题 文件大小 更新时间 录入工具 683 2024.11.13 修谱登记表 84kb 2024.11.11 千字文 15.5kb 2024.11.03 二十四孝图 1.9m 2024.11.03 中国历史年表 257kb 2017.04.26 理事会章程 18kb 2024.11.03 阴阳年转换软件下载 82kb 2017.04.26 共7条记录 页次：1/1 每页：12条记录 1 栏目导航 NAVIGATION 产品资料 其他资料 谈古论今 更多>> 杨震过洞庭湖 洞庭湖纵横800里，传说常有河神水妖作怪。有一句古谚：“斗米过洞庭，石米也是过洞庭… 2014-05-04 独拳者 独权也 杨坚为汉太尉杨震的第十四世孙。传说他的母亲吕氏在生他的前夜，曾梦见腹内苍龙盘… 2013-05-06 程门立雪 成语“程门立雪”，今用来比喻学生尊师重道的执着与专一，其典源出于宋代杨时的一段… 2013-05-06 产品展示 更多>> 世系图 检速表 苏式 苏式一 欧式 欧式横排世系 吊线后转

# 作业 十一 
### 问题（inspect_filtered_data）：4分

**(a)** 从你过滤后的数据集中随机抽取五个样本。评论它们的质量，以及它们是否适合用于语言建模，特别是考虑到我们的目标是最小化 C4 100 个领域基准测试上的困惑度。

**交付物**：来自最终过滤数据的五个随机样本。由于文档可能很长，只展示相关摘录即可。对每个样本，用 1-2 句话描述该样本以及它是否值得用于语言建模。

**(b)** 选取五个被你的过滤脚本移除和/或修改的 CC WET 样本。你的过滤过程的哪个部分移除了或修改了这些文档？你认为它们的移除和/或修改是否合理？

**交付物**：来自原始 WET 的五个随机丢弃样本。由于文档可能很长，只展示相关摘录即可。对每个样本，用 1-2 句话描述该样本以及其移除是否合理。

**(c)** 如果上述分析促使你对数据管道进行进一步修改，请在训练模型之前自由进行这些更改。报告你尝试的任何数据更改和/或迭代版本。

**交付物**：对你尝试的数据更改和/或迭代版本的描述。

---

在训练语言模型之前，我们需要对数据进行分词。使用 `transformers` 中的 GPT-2 分词器将你的过滤数据编码为整数 ID 序列用于训练语言模型。别忘了在每个文档后包含 GPT-2 的序列结束标记 `<|endoftext|>`。以下是一些入门代码，编写一个脚本，对你的过滤数据进行分词和序列化。确保按照上面的示例代码进行序列化，使用 `ids_array.tofile(output_path)`，其中 `ids_array` 是一个整数 ID 的 `np.uint16` numpy 数组。这可以确保与提供的训练脚本兼容。

你的过滤数据集中有多少个 token？

**交付物**：一个用于分词和序列化过滤数据的脚本，以及你生成的数据集中的 token 数量。



In [2]:
import multiprocessing
import os
import json
import random  # 用标准库代替 sklearn

import numpy as np
from tqdm import tqdm
from transformers import AutoTokenizer

# ==================== 配置参数 ====================
input_path = "cc_output/train_hq.jsonl"
output_dir = "tokenized"
train_output_path = os.path.join(output_dir, "train.bin")
val_output_path = os.path.join(output_dir, "val.bin")

random_seed = 42  # 随机种子

# ==================== 准备工作 ====================
os.makedirs(output_dir, exist_ok=True)
tokenizer = AutoTokenizer.from_pretrained("gpt2")

def tokenize_line_and_add_eos(line):
    return tokenizer.encode(line) + [tokenizer.eos_token_id]

# ==================== 数据加载 ====================
with open(input_path, 'r', encoding='utf-8') as f:
    lines = [json.loads(i)['text'] for i in f.readlines()]

print(f"总共加载 {len(lines)} 条文本")

# ==================== 数据集划分（9:1）===================
random.seed(random_seed)
lines_shuffled = lines.copy()
random.shuffle(lines_shuffled)

split_idx = int(len(lines_shuffled) * 0.9)
train_lines = lines_shuffled[:split_idx]
val_lines = lines_shuffled[split_idx:]

print(f"训练集: {len(train_lines)} 条 (90%)")
print(f"验证集: {len(val_lines)} 条 (10%)")

# ==================== 后续处理函数保持不变 ====================
def process_dataset(lines, desc):
    pool = multiprocessing.Pool(multiprocessing.cpu_count())
    chunksize = 100
    results = []
    
    for result in tqdm(
        pool.imap(tokenize_line_and_add_eos, lines, chunksize=chunksize),
        total=len(lines),
        desc=desc
    ):
        results.append(result)
    
    pool.close()
    pool.join()
    
    all_ids = [token_id for sublist in results for token_id in sublist]
    return all_ids

# 处理训练集
print("\n开始处理训练集...")
train_ids = process_dataset(train_lines, "Tokenizing train set")
train_array = np.array(train_ids, dtype=np.uint16)
train_array.tofile(train_output_path)
print(f"训练集已保存: {len(train_ids)} tokens")

# 处理验证集
print("\n开始处理验证集...")
val_ids = process_dataset(val_lines, "Tokenizing val set")
val_array = np.array(val_ids, dtype=np.uint16)
val_array.tofile(val_output_path)
print(f"验证集已保存: {len(val_ids)} tokens")

/home/kangkang/my_project/CS336-Chinese-co-construction/coursework/Assignment4_Data/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


总共加载 545 条文本
训练集: 490 条 (90%)
验证集: 55 条 (10%)

开始处理训练集...


Tokenizing train set:   0%|          | 0/490 [00:00<?, ?it/s]Token indices sequence length is longer than the specified maximum sequence length for this model (6533 > 1024). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (4712 > 1024). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (1451 > 1024). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (5803 > 1024). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (14984 > 1024). Running this sequence through the model will result in indexing errors
Tokenizing train s

训练集已保存: 1677854 tokens

开始处理验证集...



Tokenizing val set: 100%|██████████| 55/55 [00:00<00:00, 179.72it/s]

验证集已保存: 177093 tokens




既然我们已经对数据集进行了分词，就可以在其上训练模型了。我们将在生成的数据上训练一个 GPT-2 small 形状的模型，进行 20 万次迭代，并定期在 C4 100 个领域数据集上测量验证性能。

首先，打开位于 `cs336-basics/configs/experiment/your_data.yaml` 的配置文件，设置 `paths.train_bin` 属性指向包含你分词后训练数据的文件。你还应该设置适当的 `training.wandb_entity` 和 `training.wandb_project` 属性用于日志记录。

然后，你将使用位于 `cs336-basics/scripts/train.py` 的 `train.py` 脚本启动训练。你可以在 `cs336-basics/cs336_basics/train_config.py` 中查看将要使用的超参数。我们将使用 2 个 GPU、数据并行，每个设备的批次大小为 128。使用此配置的训练运行大约需要 7 小时完成，因此请确保预留足够的时间。在确保已设置上述提到的配置属性后，你可以使用以下命令启动训练：

```bash
uv run torchrun --standalone --nproc_per_node=2 scripts/train.py --config-name=experiment/your_data
```

再次强调，本作业的目标是优化**数据**以最小化验证损失，而不是通过修改模型和/或优化过程来尝试最小化损失，因此**不要修改**训练配置（除了上述提到的路径和 wandb 属性）或训练脚本。

在测试你的数据时，你可能还会发现将 `training.save_checkpoints` 配置参数设置为 `True` 很有帮助，这将在每次评估验证损失时保存一个检查点。这可以通过运行以下命令完成：

```bash
uv run torchrun --standalone --nproc_per_node=2 \
    scripts/train.py --config-name=experiment/your_data \
    ++training.save_checkpoints=True
```

这会将检查点保存到 `cs336-basics/output/your_data/step_N`。然后，你可以使用以下命令从保存的模型中生成样本：

```bash
uv run python scripts/generate_with_gpt2_tok.py \
    --model_path cs336-basics/output/your_data/step_N
```

---

### 问题（train_model）：2分

在你的分词数据集上训练一个语言模型（GPT-2 small 形状）。定期测量 C4 100 个领域上的验证损失（这已在 `cs336-basics/cs336_basics/train_config.py` 的配置中默认启用）。你的模型达到的最佳验证损失是多少？将此值提交到排行榜。

**交付物**：记录的最佳验证损失、相关的学习曲线，以及你所做工作的描述。

先打开`cs336-basics/configs/experiment/your_data.yaml`配置训练集和测试集的地址，这里余姚你填写地址。
wandb_entity则需要去到wandb官网进行登录后填写，它的作用是训练的时候收集数据，生成图像等，这里可以跳过，集动他，只需在训练时会有提示选择，按3即可

先cd到目录下
```bash

cd cs336-basics

```

输入命令
```bash
uv run torchrun --standalone --nproc_per_node=1 scripts/train.py --config-name=experiment/your_data
```

其中 nproc_per_node=1 参数代表有多少个gpu进行训练。